# AlphaZero data generation
_Will Kaufman, January 2021_

This is simply to generate training data so I can test different neural network designs, and to make sure it can actually learn from the data.

In [1]:
import qutip as qt
import numpy as np
import sys
import os
import multiprocessing as mp
import importlib

import torch

In [2]:
sys.path.append(os.path.abspath('..'))
import pulse_sequences as ps
import alpha_zero as az

In [3]:
# importlib.reload(az)
# importlib.reload(ps)

## Define hyperparameters

In [3]:
num_cores = 2  # 32
num_collect_initial = 2  # 5000

max_sequence_length = 48

## Define the spin system

In [4]:
delay = 1e-2  # time is relative to chemical shift strength
pulse_width = 1e-3
N = 3  # number of spins
ensemble_size = 5

In [5]:
Utarget = qt.tensor([qt.identity(2)] * N)

In [6]:
rb = az.ReplayBuffer(int(1e5))

## Get data and save to file

In [7]:
def collect_data_no_net(x):
#     print(f'collecting data without network ({x})')
    config = az.Config()
    ps_config = ps.PulseSequenceConfig(N=N, ensemble_size=ensemble_size,
                                       max_sequence_length=max_sequence_length,
                                       Utarget=Utarget,
                                       pulse_width=pulse_width, delay=delay)
    return az.make_sequence(config, ps_config, network=None, rng=ps_config.rng)

In [8]:
with mp.Pool(num_cores) as pool:
    output = pool.map(collect_data_no_net, range(num_collect_initial))
for stat in output:
    az.add_stats_to_buffer(stat, rb)

In [13]:
states, probabilities, values = zip(*rb.buffer)

In [19]:
packed_states = az.pad_and_pack(states)
probabilities = torch.cat(probabilities).view(len(rb), -1)
values = torch.cat(values).view(len(rb), -1)

In [30]:
torch.save(packed_states, 'states.pt')
torch.save(probabilities, 'probabilities.pt')
torch.save(values, 'values.pt')

In [31]:
# len(rb)